In [3]:
import os
import time 
import datetime

def print_table(data: dict[str, list[float]], precision: int = 4):
    # Convert to list of keys and find number of rows
    headers = list(data.keys())
    headers = ["index"] + headers        
    n_rows = len(next(iter(data.values())))
    data["index"] = list(range(1, n_rows+1))

    # Format floats to desired precision
    fmt = f"{{:.{precision}f}}"

    # Compute column widths
    col_widths = []
    for h in headers:
        values = data[h]
        max_val_len = max(len(fmt.format(v)) for v in values)
        col_widths.append(max(len(h), max_val_len))

    # Build header row
    header_row = " | ".join(f"{h:>{w}}" for h, w in zip(headers, col_widths))
    separator = "-+-".join("-" * w for w in col_widths)

    # Print table
    print(header_row)
    print(separator)
    for i in range(n_rows):
        row = " | ".join(fmt.format(data[h][i]).rjust(w) for h, w in zip(headers, col_widths))
        print(row)

In [4]:
def dt_to_intkey(date_str: str, time_str: str) -> float:
    dt = datetime.datetime.strptime(f"{date_str} {time_str}", "%Y_%m_%d %H_%M_%S")
    return dt.timestamp()

In [33]:
data_dir = "../../logs_cluster/logs/full_runs/21_11_2025_YCoCg_with_fixed_colorregression/results/"
res_files_unfiltered = [f for f in os.listdir(data_dir) if f.endswith(".log")]
res_files_unfiltered.sort()
# print(*res_files_unfiltered, sep="\n")
dates, times = zip(*(s.split("__")[:2] for s in res_files_unfiltered))
creation_times = {
    # key is date_time string converted to int/float for sorting
    index: dt_to_intkey(date, time)
    for index, (date, time) in enumerate(zip(dates, times))
}
only_after = dt_to_intkey("2025_10_30", "00_00_00")
res_files = [
    res_files_unfiltered[index]
    for index, creation_time in creation_times.items()
    if creation_time >= only_after
]
# print(*res_files, sep="\n")
print(len(res_files), "files found after filtering.")

24 files found after filtering.


In [34]:
supported_colorspaces = ["RGB", "YCoCg"]
results = {
}
using_image_arm = False

for f in res_files:
    with open(os.path.join(data_dir, f), "r") as infile:
        lines = infile.readlines()
        adding_to_colorspace = None
        for line in lines:
            for key in supported_colorspaces:
                if "Using color space " + key in line:
                    adding_to_colorspace = key
                    if key not in results:
                        results[key] = [[{}] for _ in range(len(res_files_unfiltered))]
                if "Using image ARM: True" in line:
                    using_image_arm = True
        if adding_to_colorspace is None:
            print(f"Could not determine color space for file {f}, skipping.")
            continue

        for line in lines:
            if "Final results after quantization" in line:
                parts = line[len("Final results after quantization:") :].strip().split(", ")
                im_index = int(f.split("_")[-1][len("kodim") :][: -len(".log")]) - 1
                for part in parts:
                    key, value = part.split(": ")
                    results[adding_to_colorspace][im_index][-1][key] = float(value)


def list_dict_to_dict_list(
    lst: list[dict[str, float]],
) -> dict[str, list[float]]:
    if not lst:
        return {}
    keys = lst[0].keys()
    dict_list = {key: [] for key in keys}
    for d in lst:
        for key in keys:
            dict_list[key].append(d.get(key, 0.0))
    return dict_list

# for key in results.keys():
#     for i, lst in enumerate(results[key]):
#         print(lst)
#         results[key][i][0] = (
#             results[key][i][0],
#         )

for key in results.keys():
    results_for_key = [d[0] for d in results[key]]
    results_for_key = list_dict_to_dict_list(results_for_key)

    print(f"{key}, Using Image ARM: {using_image_arm} Results:")
    print_table(results_for_key)
    for key, values in results_for_key.items():
        avg = sum(values) / len(values) if values else 0
        print(f"{key}: {avg:.4f}")

YCoCg, Using Image ARM: True Results:
  index |   Loss | Rate NN | Rate Latent | Rate Img
--------+--------+---------+-------------+---------
 1.0000 | 3.2632 |  0.0000 |      0.0409 |   3.2222
 2.0000 | 2.8911 |  0.0000 |      0.0602 |   2.8308
 3.0000 | 2.5164 |  0.0000 |      0.0249 |   2.4915
 4.0000 | 2.9401 |  0.0000 |      0.0300 |   2.9101
 5.0000 | 3.4628 |  0.0000 |      0.0638 |   3.3990
 6.0000 | 3.0523 |  0.0000 |      0.0792 |   2.9731
 7.0000 | 2.7263 |  0.0000 |      0.0531 |   2.6732
 8.0000 | 3.5160 |  0.0000 |      0.0703 |   3.4457
 9.0000 | 2.8563 |  0.0000 |      0.0075 |   2.8488
10.0000 | 2.9068 |  0.0000 |      0.0017 |   2.9051
11.0000 | 2.9670 |  0.0000 |      0.0227 |   2.9443
12.0000 | 2.7967 |  0.0000 |      0.0190 |   2.7778
13.0000 | 3.8010 |  0.0000 |      0.0315 |   3.7695
14.0000 | 3.2092 |  0.0000 |      0.0499 |   3.1593
15.0000 | 2.7758 |  0.0000 |      0.0236 |   2.7521
16.0000 | 2.7645 |  0.0000 |      0.0271 |   2.7374
17.0000 | 2.8476 |  0.0000